# Tokenization

### WIP: Do a spanish version: 
- https://huggingface.co/DeepESP/gpt2-spanish
- https://spacy.io/models/es/ (https://stackoverflow.com/questions/42947733/use-spacy-spanish-tokenizer)
- https://github.com/chriskhanhtran/spanish-bert
- https://www.kdnuggets.com/how-to-create-a-custom-tokenizer-for-non-english-languages-with-hugging-face-transformers



    (python) libraries
    - NLTK --> spaCy --> HuggingFace Transformers

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:


from pathlib import Path
import tiktoken #new tokenizer in the LLama models
from tiktoken.load import load_tiktoken_bpe #bpe = byte-pair encoding
import torch
import json
import matplotlib.pyplot as plt



OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\amorc\PycharmProjects\MultiModal_Llama\venvs\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

## Tiktoken tokenizer

#Tiktoken is a library for tokenizing text, specifically designed for use with OpenAI's GPT-3 and other transformer models. It provides a fast and efficient way to convert text into tokens that can be processed by these models.

Generates a vocabulary of 128000 tokens. 
    HUMAN BEINGS EXCEL IN LANGUAGE WITH JUST ABOUT 50.000!!!!
    
PARAMETERS (WHEN ENCODING WITH TIKTOKEN):

    pat_str = regex pattern that defines how text should be split into tokens
    mergeable_ranks = dict that assigns ranking to different tokens (efficient token merging) based on how frequently they appear. It is used by Byte Pair Encoding (BPE) or similar tokenization techniques to efficiently merge subwords into full words
               it's a dict with keys as strings (tokens) and values as integers (ranks):
               - Keys are byte sequences representing tokens (shown as b'...\x' because they're raw bytes that may include non-printable characters-many tokens include special characters, control codes, or UTF-8 encoded characters that don't have a simple text representation.)
               - Values are integers representing the rank/ID of each token in the vocabulary
               
                This mapping is crucial for tokenization as it determines:  
                - How text is split into tokens
                - Which byte sequences are considered single tokens
                - The priority of merging byte sequences based on their frequency in the training data
 
    special_tokens = list of special tokens that have specific meanings in the context of the model (e.g., start/end of text, padding, etc.)
    num_reserved_special_tokens = number of special tokens that are reserved for future use or specific purposes
    name = name of the tokenizer model (used for identification)
     

- COMMON WORDS LIKE 'THE ' WILL HAVE A LOWER RANKING BECAUSE THEY APPEAR FREQUENTLY AND THUS, IT IS EXPECTED THAT THEY DON´T CONVEY THAT MUCH MEANING JUST SYNTAX


In [3]:
#Initialize tiktoken tokenizer
tokenizer_path = "./content/tokenizer.model" #column of tokens in the format subword == token

num_reserved_special_tokens = 256

mergeable_ranks = load_tiktoken_bpe(tokenizer_path) 

num_base_tokens = len(mergeable_ranks)
special_tokens = [
    "<|begin_of_text|>",
    "<|end_of_text|>",
    "<|reserved_special_token_0|>",
    "<|reserved_special_token_1|>",
    "<|finetune_right_pad_id|>", #???
    "<|step_id|>",
    "<|start_header_id|>",
    "<|end_header_id|>",
    "<|eom_id|>",
    "<|eot_id|>",
    "<|python_tag|>",
]
reserved_tokens = [
    f"<|reserved_special_token_{2 + i}|>"
    for i in range(num_reserved_special_tokens - len(special_tokens))
]
special_tokens = special_tokens + reserved_tokens

# source: https://github.com/meta-llama/llama-models/blob/main/models/llama3/api/tokenizer.py#L53
tokenizer = tiktoken.Encoding(
    name=Path(tokenizer_path).name,#name of the file with the tokenizer model
    pat_str=r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+",
    mergeable_ranks=mergeable_ranks,
    special_tokens={token: len(mergeable_ranks) + i for i, token in enumerate(special_tokens)},
    #define 256 more tokens (special ones). They will be dedicated to special characters/behaviours
)





In [14]:
"""
!pip install tokenizer
import tokenizer
tokenizer.save("./content/tokenizer_model_spanish.json")
#tokenizer_path_spanish = "./content/tokenizer_model_spanish.json"
#mergeable_ranks_spanish = load_tiktoken_bpe(tokenizer_path_spanish)
"""


'\n!pip install tokenizer\nimport tokenizer\ntokenizer.save("./content/tokenizer_model_spanish.json")\n#tokenizer_path_spanish = "./content/tokenizer_model_spanish.json"\n#mergeable_ranks_spanish = load_tiktoken_bpe(tokenizer_path_spanish)\n'



```python
tokenizer_spanish = tiktoken.Encoding(
    name=Path(tokenizer_path).name + '_spanish',#name of the file with the tokenizer model
    pat_str=r"(?i:\b[a-záéíóúüñ]+['’]?[a-záéíóúüñ]*\b) | \b\d{1,3}(\.\d{3})*(,\d+)?\b |\b\d+(\.\d+)?\b |[¿¡!?,;:.\"“”(){}\[\]] |\s+",
    mergeable_ranks=mergeable_ranks,
    special_tokens={token: len(mergeable_ranks) + i for i, token in enumerate(special_tokens)},
    #define 256 more tokens (special ones). They will be dedicated to special characters/behaviours
)
```



## Regex Pattern in Spanish
r"(?i:\b[a-záéíóúüñ]+['’]?[a-záéíóúüñ]*\b) |  # Words (with optional apostrophe for foreign words)

   \b\d{1,3}(\.\d{3})*(,\d+)?\b |             # Numbers with thousands separators (1.000, 20.000, 1.234,56)
   
   \b\d+(\.\d+)?\b |                           # Numbers with decimal points (3.14, 123.45)
   
   [¿¡!?,;:.\"“”(){}\[\]] |                    # Punctuation (including Spanish-specific ¿¡)
   
   \s+"                                        # Whitespace


## Regex Pattern for token Optimization
### This regex ensures that:

    Words are properly separated.
    Numbers are split correctly.
    Punctuation is handled separately.
    Spaces and newlines are treated correctly.


r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|
- Matches common English contractions like "I'm", "you're", "he's", "we've", "it'll", "I'd".
- (?i:) makes it case-insensitive.

[^\r\n\p{L}\p{N}]?\p{L}+|
- Matches words (sequences of letters).
- \p{L} refers to Unicode letters.
- \p{N} refers to Unicode numbers.
- [^\r\n\p{L}\p{N}]? ensures that a word may have a leading punctuation character (like "hello!" still being "hello").
   
\p{N}{1,3}|
- Matches numbers of length 1 to 3 (like "1", "23", "456" but not "1234").
- This helps in efficient tokenization of numbers.

?[^\s\p{L}\p{N}]+[\r\n] * |
- Matches punctuation and symbols that are not letters or numbers.
- [^\s\p{L}\p{N}]+ ensures we capture symbols separately (e.g., "!", "@", "#$%").
- [\r\n]* allows it to optionally match line breaks.
    
\s*[\r\n]+|
- Matches newlines and surrounding spaces (\r\n refers to line breaks).
\s+(?!\S)|

- Matches whitespace sequences that are at the end of a line (avoids capturing spaces that are part of words).

\s+"
- Matches general whitespace (spaces, tabs, etc.).

In [4]:
tokenizer.encode("hello")

[15339]

In [5]:
#tokenizer_spanish.encode('hola')

NameError: name 'tokenizer_spanish' is not defined

In [12]:
tokenizer.decode([15339])

'hello'

In [13]:
tokenizer.encode("hello Andrew")

[15339, 13929]

In [14]:
tokenizer.encode("hello andrew") # = hello, and, rew

[15339, 323, 4361]

In [17]:
tokenizer.decode([323])

' and'

### Why LLM are not really reasoning with this setup?

In [25]:
question = "How many r's in the word strawberry?"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

There are 2 r's in the word "strawberry".


In [23]:
#why is it confused? 
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]

for e, d in zip(encoded_tokens, decoded_tokens):
    print(e, d)

198 

128000 <|begin_of_text|>
128006 <|start_header_id|>
882 user
128007 <|end_header_id|>
271 


4438 How
1690  many
436  r
596 's
304  in
279  the
3492  word
73700  strawberry
30 ?
128009 <|eot_id|>
128006 <|start_header_id|>
78191 assistant
128007 <|end_header_id|>
198 



In [24]:
display(HTML(html_tokens(decoded_tokens)))

In [26]:
question = "How many r's in the word s t r a w b e r r y? "
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

There are 3 r's in the word "strawberry".


In [27]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

The model was confused because its seeing 'strawberry' as a sinlge token and thus, cannot see the individual letters

## Getting the length of tokens of an input text

remember: provides charge based on tokens

In [15]:
input_text = "hello world"
len(tokenizer.encode(input_text))

2

In [16]:
question = "Who wrote the book Charlotte's Web?"
prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
len(encoded_tokens)

18

In [6]:
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
for e, d in zip(encoded_tokens, decoded_tokens):
    print(e, d)

NameError: name 'encoded_tokens' is not defined

In [18]:
from IPython.display import display, HTML
from utils import html_tokens, llama31

In [19]:
display(HTML(html_tokens(decoded_tokens)))

In [20]:
#Try one of you own:
prompt = "Supercalifragilisticexpialidocious"
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

## Llama 3.1 tokenization model file demystification

The Llama 3.1 tokenization model, named as tokenizer.model, can be downloaded along with the Llama 3.1 model weights or from the Llama models repo.

In [ ]:
# download the Llama 3.1 tokenizer model
#!wget https://raw.githubusercontent.com/meta-llama/llama-models/main/models/llama3/api/tokenizer.model

If you take a quick look at the model file, you'll see it has 128,000 lines and each line has two values separated by a space: a mysterious string and a number that starts with 0 and ends with 127,999.

In [28]:
!head -10 ./content/tokenizer.model

IQ== 0
Ig== 1
Iw== 2
JA== 3
JQ== 4
Jg== 5
Jw== 6
KA== 7
KQ== 8
Kg== 9


In [29]:
!tail -10 ./content/tokenizer.model

4LmM4LiB4Lij 127990
zrbOsQ== 127991
IOuNlOyasQ== 127992
2YjZhNin2Ko= 127993
0LLQsNGC0LjRgdGP 127994
IGvDtms= 127995
2YbYqA== 127996
INCy0YvRgdC+0LrQvtC5 127997
44O844O8 127998
6ZSm 127999


In [30]:
!wc -l ./content/tokenizer.model

128000 ./content/tokenizer.model


*Each line indeed describes one token out of 128K total tokens and its associated integer ID, and the string on each line is base64 encoded*. Use the code snippet below to decode those 128K encoded strings, and then convert the decoded bytes to more readable UTF-8 tokens.

In [31]:
import base64

encoded_tokens = []
decoded_byte_tokens = []
decoded_utf8_tokens = []

with open("./content/tokenizer.model", 'r') as file:
  for i, line in enumerate(file):
    k, v = line.strip().split(' ')
    encoded_tokens.append({k: v})
    decoded_byte_tokens.append({base64.b64decode(k): v})
    decoded_utf8_tokens.append({base64.b64decode(k).decode('utf-8', errors="replace") : v})

Let's check the first ten encoded tokens (what's stored in the tokenizer.model), and their decoded byte and UTF-8 tokens.

In [32]:
list(encoded_tokens)[:10]

[{'IQ==': '0'},
 {'Ig==': '1'},
 {'Iw==': '2'},
 {'JA==': '3'},
 {'JQ==': '4'},
 {'Jg==': '5'},
 {'Jw==': '6'},
 {'KA==': '7'},
 {'KQ==': '8'},
 {'Kg==': '9'}]

In [33]:
list(decoded_byte_tokens)[:10]

[{b'!': '0'},
 {b'"': '1'},
 {b'#': '2'},
 {b'$': '3'},
 {b'%': '4'},
 {b'&': '5'},
 {b"'": '6'},
 {b'(': '7'},
 {b')': '8'},
 {b'*': '9'}]

In [34]:
list(decoded_utf8_tokens)[:10]

[{'!': '0'},
 {'"': '1'},
 {'#': '2'},
 {'$': '3'},
 {'%': '4'},
 {'&': '5'},
 {"'": '6'},
 {'(': '7'},
 {')': '8'},
 {'*': '9'}]

Let's confirm the tokenizer.model file stores the base64 encoded strings for tokens, e.g. the token "hello".

In [35]:
base64.b64encode('h'.encode('utf-8'))

b'aA=='

In [36]:
base64.b64encode('hello'.encode('utf-8'))

b'aGVsbG8='

In [37]:
!grep "aGVsbG8=" ./content/tokenizer.model

aGVsbG8= 15339


# More LLM reasoning vs tokenization

Let's try out Llama 3.1 on some recent tokenization related LLM problems, and see if we can improve its reasoning by some prompt engineering.

## Simple math problem

In [38]:
question = "Which number is bigger, 9.11 or 9.9? "
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

9.9 is bigger.


In [39]:
response = llama31(prompt, 70)
print(response)

9.9 is bigger than 9.11.


In [40]:
response = llama31(prompt, 405)
print(response)

The number 9.11 is bigger than 9.9.

To compare these numbers, you can look at the decimal part. Since 0.11 is greater than 0.09 (or 0.9 - 0.9 = 0 and 0.11 - 0.09 = 0.02), 9.11 is greater than 9.9


Somehow the largest Llama 3.1 405b model returns the incorrect result. From the visualization of the tokens in the prompt, you can see the number 9.11 is split into 3 tokens: "9", ".", and ".11", while 9.9 into 2 tokens: "9", ".", "9". If the two numbers are encoded as the two numbers themselves, correct model response will be more likely.

In [41]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
[x for x in zip(encoded_tokens, decoded_tokens)]

[(198, '\n'),
 (128000, '<|begin_of_text|>'),
 (128006, '<|start_header_id|>'),
 (882, 'user'),
 (128007, '<|end_header_id|>'),
 (271, '\n\n'),
 (23956, 'Which'),
 (1396, ' number'),
 (374, ' is'),
 (11493, ' bigger'),
 (11, ','),
 (220, ' '),
 (24, '9'),
 (13, '.'),
 (806, '11'),
 (477, ' or'),
 (220, ' '),
 (24, '9'),
 (13, '.'),
 (24, '9'),
 (30, '?'),
 (220, ' '),
 (128009, '<|eot_id|>'),
 (128006, '<|start_header_id|>'),
 (78191, 'assistant'),
 (128007, '<|end_header_id|>'),
 (198, '\n')]

In [42]:
display(HTML(html_tokens(decoded_tokens)))

## String reversing

First, for a common word "amazing", all 3 Llama 3.1 chat models reverse the string correctly.

In [ ]:
input = "Reverse the string 'amazing'"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

In [ ]:
response = llama31(prompt, 70)
print(response)

In [ ]:
response = llama31(prompt, 405)
print(response)

In [ ]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

For a less common word "language", Llama 3.1 8B doesn't return the correct result, but 70B and 405B do.

In [ ]:
input = "Reverse the string 'language'"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

In [ ]:
response = llama31(prompt, 70)
print(response)

In [ ]:
response = llama31(prompt, 405)
print(response)

In [43]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

For the string "XMLElement", none of the 3 models is correct.
### WHY???

In [44]:
input = "Reverse the string 'XMLElement'"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

The reverse of 'XMLElement' is 'deMltxE'.


In [45]:
response = llama31(prompt, 70)
print(response)

'tnemELX'


In [46]:
response = llama31(prompt, 405)
print(response)


The reverse of 'XMLElement' is 'TNEMELEMX'.


In [47]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))